# Compare k-mer abundance and presence/absence vs "traditional" single-cell RNA-seq processing

- 500 k-mers, with abundance, were hashed from ~50k cells in Tabula Muris using `sourmash`


## Load Tabula Muris Senis data with fixed annotations

In [2]:
# import scanpy.api as sc
# sc.logging.print_versions()

# h5ad = '/home/olga/ibm_lg/tabula-muris-senis/maca-facs-re-processed-annotated-fixed-tissues.h5ad'

# tms = sc.read_h5ad(h5ad)
# tms

In [3]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

import khtools

%load_ext autoreload
%autoreload 2

## Read 50k x 50k similarity matrix (~50GB csv file)

In [4]:
csv = '/home/olga/pureScratch/olgabot-maca/facs/sourmash_compare_k31_ignore_abundance.csv'
! ls -lha $csv

-rw-rw-r-- 1 olga olga 14G Aug  7 11:51 /home/olga/pureScratch/olgabot-maca/facs/sourmash_compare_k31_ignore_abundance.csv


In [5]:
%time similarity = dd.read_csv(csv, sample=2048000)

CPU times: user 6min 55s, sys: 3.16 s, total: 6min 58s
Wall time: 6min 57s


In [6]:
similarity.shape

(Delayed('int-7e2deed7-b539-4291-8a62-ed18fcee950c'), 50312)

Had to increase `sample` to be the number of bytes(?) per row for `dd.read_csv` to be happy

In [7]:
# similarity.dtypes

In [ ]:
# %time similarity = dd.read_csv(csv, sample=2048000)

In [ ]:
# similarity_pd = similarity.to_

In [ ]:
%time similarity.iloc[:, :5].head()

In [ ]:
# similarity.to_dask_array?

In [ ]:
similarity_da = similarity.to_dask_array(lengths=True)

In [ ]:
similarity_da

In [29]:
# Can't save to hdf5 because there's a limit of 2000 columns
# https://stackoverflow.com/q/16639503/1628971

# similarity.to_hdf(csv.replace(".csv", '.hdf5'), 'data')

## Dimensionality Reduction

### Run PCA on jaccard

In [33]:
from dask_ml.decomposition import PCA

pca = PCA(n_components=50, copy=False, svd_solver='randomized')

similarities_pca = pca.fit_transform(similarity_da)
# print(similarities_pca.shape)

ValueError: n_components=nan must be between 0 and min(n_samples, n_features)=nan with svd_solver='full'

In [14]:
annotations = pd.read_csv('https://github.com/czbiohub/tabula-muris/raw/master/00_data_ingest/18_global_annotation_csv/annotations_facs.csv', 
                          index_col='cell')
annotations.index = annotations.index.str.replace('.', '-')
annotations.columns = annotations.columns.str.replace('.', '_')
annotations['sample_id'] = annotations.index
annotations = annotations.fillna("NA")
print(annotations.shape)
annotations.head()

(44949, 24)


/home/olga/miniconda3/envs/kmer-hashing/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Neurog3>0_raw,Neurog3>0_scaled,cell_ontology_class,cell_ontology_id,cluster_ids,free_annotation,mouse_id,mouse_sex,plate_barcode,subsetA,...,subsetC_cluster_ids,subsetD,subsetD_cluster_ids,subsetE,subsetE_cluster_ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2,sample_id
cell,,,,,,,,,,,,,,,,,,,,,
A1-B000610-3_56_F-1-1,NA,NA,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_56_F,F,B000610,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-7.182841,-10.895094,A1-B000610-3_56_F-1-1
A1-B002764-3_38_F-1-1,NA,NA,bladder urothelial cell,CL:1001428,3,Luminal bladder epithelial cell,3_38_F,F,B002764,NA,...,NA,NA,NA,NA,NA,NA,Bladder,4.199059,-11.050240,A1-B002764-3_38_F-1-1
A1-B002771-3_39_F-1-1,NA,NA,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_39_F,F,B002771,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-11.995435,-7.325534,A1-B002771-3_39_F-1-1
A1-D041914-3_8_M-1-1,NA,NA,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_8_M,M,D041914,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-6.820022,-14.174246,A1-D041914-3_8_M-1-1
A1-D042253-3_9_M-1-1,NA,NA,bladder cell,CL:1001319,2,Bladder mesenchymal cell,3_9_M,M,D042253,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-24.163538,-7.499349,A1-D042253-3_9_M-1-1


In [15]:
color_cols = ['tissue', 'subtissue', 'cell_ontology_class', 'mouse_id', 'mouse_sex', 'plate_barcode']
palette_names = ['tab20', 'tab10', 'cubehelix', 'husl', 'Paired', 'Set2']
palettes = dict(zip(color_cols, palette_names))
palettes

{'tissue': 'tab20',
 'subtissue': 'tab10',
 'cell_ontology_class': 'cubehelix',
 'mouse_id': 'husl',
 'mouse_sex': 'Paired',
 'plate_barcode': 'Set2'}

In [ ]:
graphs = {}
positions = {}

### Build KNN on similarities

In [ ]:
%%time

key = 'k31_without_abundance'

n_neighbors = 5
figure_folder = 's3://kmer-hashing/tabula-muris/n_hashes=500/'
figure_prefix = 'molecule=DNA_ksize=31_ignore-abundance=False'
title = 'Tabula Muris'

g, pos = khtools.knn.build_graph_and_plot(similarity, annotations, n_neighbors,
                                          color_cols, palettes, figure_folder, figure_prefix, title)

graphs[key] = g
positions[key] = pos

In [27]:
khtools.__version__

'0.1.0'

## Run Leiden Clustering


Below is excerpted from [scanpy's leiden clustering algorithm](https://github.com/theislab/scanpy/blob/master/scanpy/tools/_leiden.py)

In [6]:


try:
    import leidenalg
except ImportError:
    raise ImportError('Please install the leiden algorithm: `pip3 install leidenalg`.')

# convert it to igraph
g = scanpy.utils.get_igraph_from_adjacency(adjacency, directed=directed)
# flip to the default partition type if not overriden by the user
if partition_type is None:
    partition_type = leidenalg.RBConfigurationVertexPartition
# prepare find_partition arguments as a dictionary, appending to whatever the user provided
# it needs to be this way as this allows for the accounting of a None resolution
# (in the case of a partition variant that doesn't take it on input)

partition_kwargs = None

if partition_kwargs is None:
    partition_kwargs = {}
if use_weights:
    partition_kwargs['weights'] = np.array(g.es['weight']).astype(np.float64)
partition_kwargs['n_iterations'] = n_iterations
partition_kwargs['seed'] = random_state
if resolution is not None:
    partition_kwargs['resolution_parameter'] = resolution
# clustering proper
part = leidenalg.find_partition(g, partition_type, **partition_kwargs)
# store output into adata.obs
groups = np.array(part.membership)


ImportError: Please install the leiden algorithm: `pip3 install leidenalg`.

## Make confusion matrix with Tabula Muris Senis leiden clustering

In [ ]:
tms.obs['leiden'].head()